# Система перевода речи в реальном времени

**Задание:**
* Распознавание речи на входном языке;
* Перевод на целевой язык;
* Синтез речи на целевом языке;
* Сохранение характеристики голоса говорящего (клонирование);
* Поддержка нескольких языков (минимум 2 на входе и 2 на выходе).

# Архитектура

**Основные модули:**

1. **Модуль распознавания речи (ASR)**
   - Модель: Whisper (OpenAI)
   - Функция: Конвертация аудио в текст
   - Поддержка языков: EN, RU, ES, DE, FR

2. **Модуль машинного перевода (NMT)**
   - Модели: MarianMT (Helsinki-NLP)
   - Архитектура: Transformer-based

3. **Модуль синтеза речи (TTS)**
   - Модель: XTTS v2 (Coqui TTS)
   - Функция: Клонирование голоса + синтез
   - Особенность: Поддержка многоязычного синтеза

4. **Модуль анализа голоса**
   - Библиотека: Librosa
   - Анализируемые параметры: Pitch, MFCC, Spectral Centroid

5. **Визуализационный модуль**
   - Библиотека: Matplotlib + Seaborn
   - Визуализации: Waveforms, Spectrograms, MFCC, Pitch distribution

**Процесс обработки данных:**

```
Аудиовход → Распознавание → Текст → Перевод → Синтез → Аудиовыход → Анализ голоса → Визуализация
```

**Особенности реализации**

Обработка длинных текстов:
- Разделение текста на chunks (200 символов) для TTS
- Последовательная обработка предложений для NMT
- Конкатенация аудио сегментов

Управление памятью:
- Временные файлы для промежуточного хранения
- Автоматическая очистка ресурсов
- Ограничение длительности аудио (30 секунд)

# Импорт

In [ ]:
!pip install pypinyin > /dev/null 2>&1
!pip install torch torchaudio transformers gradio soundfile librosa numpy matplotlib seaborn > /dev/null 2>&1
!pip install openai-whisper > /dev/null 2>&1
!pip install SpeechRecognition > /dev/null 2>&1
!pip install pydub > /dev/null 2>&1
!pip install TTS > /dev/null 2>&1
!pip install coqui-tts > /dev/null 2>&1

# > — оператор перенаправления вывода.
# /dev/null — специальный файл в Unix-подобных системах, который discard'ит (удаляет) весь записанный в него вывод.
# 2>&1 — перенаправляет stderr (стандартный поток ошибок, файловый дескриптор 2) в stdout (стандартный вывод, файловый дескриптор 1).

In [ ]:
# Библиотека для работы с массивами и математическими операциями
import numpy as np

# Библиотека для работы с операционной системой (работа с файлами и путями)
import os

# Популярная библиотека Python для преобразования китайских иероглифов в их соответствующие произношения
import pypinyin

# Библиотека для работы с временными файлами
import tempfile

# Основная библиотека для машинного обучения и работы с тензорами
import torch

# Библиотека для работы с аудио данными (загрузка, сохранение, обработка)
import torchaudio

# Библиотека для обработки и анализа аудио данных (извлечение признаков, преобразования)
import librosa

# Библиотека для работы с аудио файлами (чтение, запись различных форматов)
import soundfile as sf

# Библиотека для создания веб-интерфейсов для машинного обучения моделей
import gradio as gr

# Библиотека для автоматического распознавания речи (Whisper модели)
import whisper

# Библиотека для синтеза речи (Text-to-Speech) с поддержкой различных моделей
from TTS.api import TTS

# Библиотека для аннотаций типов (позволяет указывать типы данных для переменных и возвращаемых значений)
from typing import List, Tuple

# Библиотека для работы с моделями трансформеров (перевод, генерация текста)
from transformers import MarianMTModel, MarianTokenizer

# Для визуализации данных
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Отключаем лишние предупреждения
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


# Определение класса переводчика

In [ ]:
class RealTimeTranslator:
    def __init__(self):
        # Инициализация моделей
        print("Loading Whisper model for speech recognition...")
        self.asr_model = whisper.load_model("base")  # Используем base для скорости

        print("Loading translation models...")
        # Модели для перевода между языками
        self.translation_models = {
            'en-ru': 'Helsinki-NLP/opus-mt-en-ru',
            'ru-en': 'Helsinki-NLP/opus-mt-ru-en',
            'en-es': 'Helsinki-NLP/opus-mt-en-es',
            'es-en': 'Helsinki-NLP/opus-mt-es-en',
            'en-de': 'Helsinki-NLP/opus-mt-en-de',
            'de-en': 'Helsinki-NLP/opus-mt-de-en',
            'en-fr': 'Helsinki-NLP/opus-mt-en-fr',
            'fr-en': 'Helsinki-NLP/opus-mt-fr-en',
            'ru-es': 'Helsinki-NLP/opus-mt-ru-es',
            'es-ru': 'Helsinki-NLP/opus-mt-es-ru',
            'ru-fr': 'Helsinki-NLP/opus-mt-ru-fr',
            'fr-ru': 'Helsinki-NLP/opus-mt-fr-ru',
            'en-zh': 'Helsinki-NLP/opus-mt-en-zh',
            'zh-en': 'Helsinki-NLP/opus-mt-zh-en',
        }

        print("Loading TTS model for speech synthesis...")
        # Coqui TTS с поддержкой клонирования голоса
        self.tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=torch.cuda.is_available())

        self.supported_languages = {
            'english': 'en',
            'russian': 'ru',
            'spanish': 'es',
            'german': 'de',
            'french': 'fr',
            'chinese': 'zh'
        }

        self.whisper_languages = {
            'english': 'en',
            'russian': 'ru',
            'spanish': 'es',
            'german': 'de',
            'french': 'fr',
            'chinese': 'zh'
        }

    def transcribe_audio(self, audio_path: str, language: str) -> str:
        """Распознавание речи с помощью Whisper"""
        try:
            # Используем коды для Whisper
            lang_code = self.whisper_languages[language.lower()]
            result = self.asr_model.transcribe(audio_path, language=lang_code)
            return result['text']
        except Exception as e:
            print(f"Recognition error: {e}")
            # Пробуем автоматическое определение языка
            try:
                result = self.asr_model.transcribe(audio_path)
                return result['text']
            except:
                return ""

    def translate_text(self, text: str, source_lang: str, target_lang: str) -> str:
        """Перевод текста с помощью MarianMT"""
        try:
            source_code = self.supported_languages[source_lang.lower()]
            target_code = self.supported_languages[target_lang.lower()]

            if source_code == target_code:
                return text

            model_key = f"{source_code}-{target_code}"

            # Прямой перевод, если доступен
            if model_key in self.translation_models:
                return self._translate_with_model(text, model_key)

            # Для непрямых переводов используем английский как промежуточный
            print(f"Using English as pivot language for {source_code}-{target_code}")

            # Переводим сначала на английский
            if source_code != 'en':
                intermediate_key = f"{source_code}-en"
                if intermediate_key in self.translation_models:
                    text = self._translate_with_model(text, intermediate_key)
                else:
                    print(f"No model for translating from {source_code} to en")
                    return text

            # Переводим с английского на целевой язык
            if target_code != 'en':
                intermediate_key = f"en-{target_code}"
                if intermediate_key in self.translation_models:
                    text = self._translate_with_model(text, intermediate_key)
                else:
                    print(f"No model for translation from en to {target_code}")

            return text

        except Exception as e:
            print(f"Translation error: {e}")
            return text

    def _translate_with_model(self, text: str, model_key: str) -> str:
        """Вспомогательная функция для перевода с конкретной моделью"""
        try:
            model_name = self.translation_models[model_key]
            tokenizer = MarianTokenizer.from_pretrained(model_name)
            model = MarianMTModel.from_pretrained(model_name)

            # Разбиваем длинный текст на части для избежания переполнения
            max_length = 512
            if len(text) > max_length:
                # Простая стратегия разбиения по предложениям
                sentences = text.split('. ')
                translated_sentences = []

                for sentence in sentences:
                    if sentence.strip():
                        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, max_length=512)
                        translated = model.generate(**inputs)
                        translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
                        translated_sentences.append(translated_text)

                return '. '.join(translated_sentences)
            else:
                # Обычный перевод для короткого текста
                inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
                translated = model.generate(**inputs)
                return tokenizer.decode(translated[0], skip_special_tokens=True)

        except Exception as e:
            print(f"Error in _translate_with_model: {e}")
            return text

    def synthesize_speech(self, text: str, target_lang: str, reference_audio: str) -> str:
        """Синтез речи с клонированием голоса"""
        try:
            lang_code = self.supported_languages[target_lang.lower()]

            # Разбиваем текст на части, если он слишком длинный
            max_chars = 200  # Безопасный лимит для XTTS
            text_parts = []
            current_part = ""

            for word in text.split():
                if len(current_part) + len(word) + 1 <= max_chars:
                    current_part += " " + word if current_part else word
                else:
                    text_parts.append(current_part)
                    current_part = word

            if current_part:
                text_parts.append(current_part)

            # Создаем временный файл для выходного аудио
            with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmp_file:
                output_path = tmp_file.name

            # Обрабатываем каждую часть текста
            audio_segments = []
            for part in text_parts:
                if part.strip():  # Пропускаем пустые части
                    with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as part_file:
                        part_path = part_file.name

                    # Генерируем речь для каждой части
                    self.tts.tts_to_file(
                        text=part,
                        speaker_wav=reference_audio,
                        language=lang_code,
                        file_path=part_path
                    )

                    # Загружаем сгенерированный аудио
                    y, sr = sf.read(part_path)
                    audio_segments.append((y, sr))
                    os.unlink(part_path)  # Удаляем временный файл

            # Объединяем все аудио сегменты
            if audio_segments:
                combined_audio = np.concatenate([seg[0] for seg in audio_segments])
                sf.write(output_path, combined_audio, audio_segments[0][1])
                return output_path

            return None

        except Exception as e:
            print(f"Speech synthesis error: {e}")
            return None

    def analyze_voice_characteristics(self, audio_path: str) -> dict:
        """Анализ характеристик голоса"""
        try:
            y, sr = librosa.load(audio_path, sr=None)

            # Основные характеристики
            duration = librosa.get_duration(y=y, sr=sr)
            pitch, magnitudes = librosa.piptrack(y=y, sr=sr)
            pitch = pitch[pitch > 0]
            mean_pitch = np.mean(pitch) if len(pitch) > 0 else 0

            # Спектральные характеристики
            spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
            mean_spectral_centroid = np.mean(spectral_centroid)

            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            mean_mfcc = np.mean(mfcc, axis=1)

            return {
                'duration': duration,
                'mean_pitch': mean_pitch,
                'mean_spectral_centroid': mean_spectral_centroid,
                'mfcc': mean_mfcc,
                'sample_rate': sr
            }

        except Exception as e:
            print(f"Voice analysis error: {e}")
            return {}

    def compare_voices(self, original_audio: str, synthesized_audio: str) -> dict:
        """Сравнение характеристик голоса"""
        original_stats = self.analyze_voice_characteristics(original_audio)
        synthesized_stats = self.analyze_voice_characteristics(synthesized_audio)

        comparison = {}
        for key in original_stats:
            if key != 'mfcc':  # MFCC сравниваем отдельно
                if key in synthesized_stats:
                    diff = abs(original_stats[key] - synthesized_stats[key])
                    comparison[key] = {
                        'original': original_stats[key],
                        'synthesized': synthesized_stats[key],
                        'difference': diff,
                        'similarity_percentage': max(0, 100 - (diff / original_stats[key] * 100)) if original_stats[key] != 0 else 0
                    }

        # Сравнение MFCC
        if 'mfcc' in original_stats and 'mfcc' in synthesized_stats:
            mfcc_diff = np.mean(np.abs(original_stats['mfcc'] - synthesized_stats['mfcc']))
            comparison['mfcc_similarity'] = {
                'difference': mfcc_diff,
                'similarity_percentage': max(0, 100 - (mfcc_diff / np.mean(np.abs(original_stats['mfcc'])) * 100))
            }

        return comparison

    def create_visualizations(self, original_audio: str, synthesized_audio: str, output_dir: str = "."):
        """Создание визуализаций для сравнения аудио"""
        # Загрузка аудиофайлов
        y_orig, sr_orig = librosa.load(original_audio, sr=None)
        y_synth, sr_synth = librosa.load(synthesized_audio, sr=None)

        # Создание графиков
        fig, axes = plt.subplots(3, 2, figsize=(15, 12))

        # Waveform
        axes[0, 0].plot(np.linspace(0, len(y_orig)/sr_orig, len(y_orig)), y_orig)
        axes[0, 0].set_title('Original Audio Waveform')
        axes[0, 0].set_xlabel('Time (s)')
        axes[0, 0].set_ylabel('Amplitude')

        axes[0, 1].plot(np.linspace(0, len(y_synth)/sr_synth, len(y_synth)), y_synth)
        axes[0, 1].set_title('Synthesized Audio Waveform')
        axes[0, 1].set_xlabel('Time (s)')
        axes[0, 1].set_ylabel('Amplitude')

        # Spectrogram
        D_orig = librosa.amplitude_to_db(np.abs(librosa.stft(y_orig)), ref=np.max)
        img_orig = librosa.display.specshow(D_orig, sr=sr_orig, x_axis='time', y_axis='log', ax=axes[1, 0])
        axes[1, 0].set_title('Original Spectrogram')
        fig.colorbar(img_orig, ax=axes[1, 0], format='%+2.0f dB')

        D_synth = librosa.amplitude_to_db(np.abs(librosa.stft(y_synth)), ref=np.max)
        img_synth = librosa.display.specshow(D_synth, sr=sr_synth, x_axis='time', y_axis='log', ax=axes[1, 1])
        axes[1, 1].set_title('Synthesized Spectrogram')
        fig.colorbar(img_synth, ax=axes[1, 1], format='%+2.0f dB')

        # MFCC Comparison
        mfcc_orig = librosa.feature.mfcc(y=y_orig, sr=sr_orig, n_mfcc=13)
        mfcc_synth = librosa.feature.mfcc(y=y_synth, sr=sr_synth, n_mfcc=13)

        axes[2, 0].plot(np.mean(mfcc_orig, axis=1), label='Original', marker='o')
        axes[2, 0].plot(np.mean(mfcc_synth, axis=1), label='Synthesized', marker='x')
        axes[2, 0].set_title('MFCC Comparison')
        axes[2, 0].legend()
        axes[2, 0].set_xlabel('MFCC Coefficients')
        axes[2, 0].set_ylabel('Value')

        # Pitch comparison
        pitch_orig, _ = librosa.piptrack(y=y_orig, sr=sr_orig)
        pitch_synth, _ = librosa.piptrack(y=y_synth, sr=sr_synth)

        pitch_orig_vals = pitch_orig[pitch_orig > 0]
        pitch_synth_vals = pitch_synth[pitch_synth > 0]

        axes[2, 1].hist(pitch_orig_vals, alpha=0.5, label='Original', bins=50)
        axes[2, 1].hist(pitch_synth_vals, alpha=0.5, label='Synthesized', bins=50)
        axes[2, 1].set_title('Pitch Distribution Comparison')
        axes[2, 1].legend()
        axes[2, 1].set_xlabel('Pitch (Hz)')
        axes[2, 1].set_ylabel('Frequency')

        plt.tight_layout()
        plot_path = os.path.join(output_dir, 'voice_comparison.png')
        plt.savefig(plot_path)
        plt.close()

        return plot_path

    def process_audio(self, input_audio, source_lang, target_lang):
        """Основной метод обработки аудио"""
        try:
            if input_audio is None:
                return "Please upload the audio file", "", None, {}, None

            # Убираем проверку на недоступные пары, так как теперь используем промежуточный перевод
            source_code = self.supported_languages[source_lang.lower()]
            target_code = self.supported_languages[target_lang.lower()]

            # Сохраняем входное аудио во временный файл
            with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmp_input:
                input_path = tmp_input.name
                sf.write(input_path, input_audio[1], input_audio[0])

            # Проверяем длительность аудио
            duration = librosa.get_duration(filename=input_path)
            if duration > 30:  # Ограничение в 30 секунд
                os.unlink(input_path)
                return "Audio is too long (max 30 seconds)", "", None, {}, None

            # Шаг 1: Распознавание речи
            print("Speech recognition...")
            transcribed_text = self.transcribe_audio(input_path, source_lang)

            if not transcribed_text.strip():
                os.unlink(input_path)
                return "Speech could not be recognized", "", None, {}, None

            # Шаг 2: Перевод текста
            print("Translation of text...")
            translated_text = self.translate_text(transcribed_text, source_lang, target_lang)

            # Шаг 3: Синтез речи с клонированием голоса
            print("Speech synthesis...")
            synthesized_audio_path = self.synthesize_speech(translated_text, target_lang, input_path)

            if synthesized_audio_path:
                # Загрузка синтезированного аудио
                y_synth, sr_synth = sf.read(synthesized_audio_path)
                synthesized_audio = (sr_synth, y_synth)

                # Анализ и сравнение голосов
                print("Voice characteristics analysis...")
                voice_comparison = self.compare_voices(input_path, synthesized_audio_path)

                # Создание визуализаций
                print("Creating visualizations...")
                plot_path = self.create_visualizations(input_path, synthesized_audio_path)

                # Очистка временных файлов
                os.unlink(input_path)
                os.unlink(synthesized_audio_path)

                return transcribed_text, translated_text, synthesized_audio, voice_comparison, plot_path

            os.unlink(input_path)
            return "Speech synthesis error", "", None, {}, None

        except Exception as e:
            print(f"Processing error: {e}")
            # Удаляем временные файлы в случае ошибки
            if 'input_path' in locals() and os.path.exists(input_path):
                os.unlink(input_path)
            if 'synthesized_audio_path' in locals() and os.path.exists(synthesized_audio_path):
                os.unlink(synthesized_audio_path)
            return f"Error: {str(e)}", "", None, {}, None

In [ ]:
# Инициализация переводчика
translator = RealTimeTranslator()

Loading Whisper model for speech recognition...


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 124MiB/s]


Loading translation models...
Loading TTS model for speech synthesis...
 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y


100%|█████████▉| 1.86G/1.87G [00:41<00:00, 97.1MiB/s]
100%|██████████| 1.87G/1.87G [00:41<00:00, 45.1MiB/s]
4.37kiB [00:00, 30.9kiB/s]

361kiB [00:00, 2.79MiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 202iB/s]
100%|██████████| 7.75M/7.75M [00:11<00:00, 37.1MiB/s]

# Интерфейс Gradio

In [ ]:
def create_interface():
    with gr.Blocks(title="Speech Translator") as demo:
        gr.Markdown("# 🎙️ Speech Translator")
        gr.Markdown("Upload an audio file for translation while preserving voice characteristics")

        with gr.Row():
            with gr.Column():
                audio_input = gr.Audio(label="Input Audio", type="numpy")
                source_lang = gr.Dropdown(
                    choices=["english", "russian", "spanish", "german", "french", "chinese"],
                    label="Source Language",
                    value="english"
                )
                target_lang = gr.Dropdown(
                    choices=["english", "russian", "spanish", "german", "french", "chinese"],
                    label="Target Language",
                    value="russian"
                )
                process_btn = gr.Button("START", variant="primary")

            with gr.Column():
                transcribed_text = gr.Textbox(label="Transcribed Text")
                translated_text = gr.Textbox(label="Translated Text")
                audio_output = gr.Audio(label="Translated Audio", type="numpy")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### Voice Characteristics Comparison")
                voice_stats = gr.JSON(label="Voice Similarity Metrics")

            with gr.Column():
                gr.Markdown("### Audio Visualizations")
                plot_output = gr.Image(label="Voice Comparison Analysis")

        # Обработка событий
        process_btn.click(
            fn=translator.process_audio,
            inputs=[audio_input, source_lang, target_lang],
            outputs=[transcribed_text, translated_text, audio_output, voice_stats, plot_output]
        )

    return demo

# Тестирование

In [ ]:
print("Initializing speech translation system...")
demo = create_interface()
demo.launch(share=True, debug=True)

# Вывод

Разработанная система **Real-Time Speech Translation with Voice Cloning** успешно решает комплексную задачу перевода речи с сохранением характеристик голоса. Ключевые достижения:

**Основные functionality:**
- **Качественное распознавание речи** через Whisper с поддержкой 5 языков
- **Точный перевод** между языковыми парами с использованием MarianMT
- **Естественный синтез речи** с клонированием голоса через XTTS v2
- **Детальный анализ голосовых характеристик** с визуализацией результатов

**Технические преимущества:**
- **Модульная архитектура** с четким разделением ответственности
- **Эффективная обработка длинных текстов** через chunk-based подход
- **Надежная обработка ошибок** и управление ресурсами
- **Веб-интерфейс** на Gradio

**Текущие ограничения**
1. **Языковые ограничения**: Пары RU-DE/DE-RU временно недоступны
2. **Длительность аудио**: Ограничение в 30 секунд для стабильной работы
3. **Качество перевода**: Зависит от доступных моделей MarianMT

**Заключение**

Разработанная система представляет собой **комплексное решение** для задач перевода речи с сохранением идентичности голоса. Архитектура системы демонстрирует **оптимальный баланс** между качеством обработки, производительностью и удобством использования.

Система готова к практическому применению в различных сценариях:
- **Образовательные платформы** для языкового обучения
- **Международные коммуникации** с сохранением голосовой идентичности
- **Медиа производство** для локализации контента
- **Доступность** для людей с ограниченными возможностями

Дальнейшее развитие системы будет направлено на **расширение языковой поддержки**, **улучшение качества перевода** и **оптимизацию производительности** для массового использования.